In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import torch

import gymnasium as gym

from models.dqn import DQN
from rl_models.replay_buffer import ReplayBuffer

In [3]:
env = gym.make("CartPole-v0", render_mode="rgb_array").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]

n_actions, state_dim

/home/evgeny/anaconda3/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


(2, 4)

In [4]:
agent = DQN(state_dim, n_actions, lr=1e-4)
buffer = ReplayBuffer(10000)

In [5]:
def generate_session(env, t_max=1000, epsilon=0.0, train=False):
    total_reward = 0
    s, _ = env.reset()

    for t in range(t_max):
        s = torch.tensor(s, dtype=torch.float)
        
        if train and np.random.rand() < epsilon:
            a = torch.tensor(env.action_space.sample())
        else:
            a = agent.choose_action(s)
        
        next_s, r, done, _, _ = env.step(a.numpy())
        
        next_s = torch.tensor(next_s, dtype=torch.float)
        r = torch.tensor(r, dtype=torch.float)
        batch = [[[s], [a], [r], [next_s], [done]]]
        if train:
            agent.learning_step(batch)
            agent.update_network_parameters()

        total_reward += r
        s = next_s
        if done:
            break

    return total_reward

In [6]:
epsilon = 0.5

for i in range(1000):
    session_rewards = [generate_session(env, t_max=1000, epsilon=epsilon, train=True) for _ in range(100)]
    print("epoch #{}\tmean reward = {:.3f}\t".format(i, np.mean(session_rewards)))

    epsilon *= 0.99
    if np.mean(session_rewards) > 300:
        print("You Win!")
        break

/tmp/ipykernel_76876/681407464.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float)


epoch #0	mean reward = 13.890	
epoch #1	mean reward = 14.580	
epoch #2	mean reward = 14.810	
epoch #3	mean reward = 24.400	
epoch #4	mean reward = 30.130	
epoch #5	mean reward = 37.730	
epoch #6	mean reward = 57.080	
epoch #7	mean reward = 62.310	
epoch #8	mean reward = 101.540	
epoch #9	mean reward = 117.190	
epoch #10	mean reward = 143.930	
epoch #11	mean reward = 139.410	
epoch #12	mean reward = 152.040	
epoch #13	mean reward = 159.850	
epoch #14	mean reward = 171.330	
epoch #15	mean reward = 196.840	
epoch #16	mean reward = 212.540	
epoch #17	mean reward = 222.070	
epoch #18	mean reward = 248.810	
epoch #19	mean reward = 319.800	
You Win!
